In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Aggregating Data 

The `groupby()` function aggregates data over multiple rows of a DataFrame:
* It can find the sum of a column
* It can get the mean of a subset of values in a column.

`groupby()` returns a GroupBy object that works with aggregate functions.

In [21]:
## Begin Example
file = "../data/opsd_germany_daily.csv"

data = pd.read_csv(file)

# Convert to date time
data["Date"] = pd.to_datetime(data["Date"])

# Set index


data.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4383 entries, 0 to 4382
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         4383 non-null   datetime64[ns]
 1   Consumption  4383 non-null   float64       
 2   Wind         2920 non-null   float64       
 3   Solar        2188 non-null   float64       
 4   Wind+Solar   2187 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 171.3 KB


In [25]:
start_date = "2012-12-01"
end_date = "2012-12-31"

filtered = data[(data.index >= start_date) & (data.index <= end_date)]
filtered["date"] = filtered["Date"].dt.date



TypeError: Invalid comparison between dtype=int64 and str

In [18]:
grouped_by_date = filtered.groupby("Date")["Consumption"].sum()
grouped_by_date

Date
2012-12-01    1243.897
2012-12-02    1151.585
2012-12-03    1484.956
2012-12-04    1469.941
2012-12-05    1507.404
2012-12-06    1502.183
2012-12-07    1519.663
2012-12-08    1301.548
2012-12-09    1156.396
2012-12-10    1515.692
2012-12-11    1520.748
2012-12-12    1566.704
2012-12-13    1554.672
2012-12-14    1468.353
2012-12-15    1220.238
2012-12-16    1132.311
2012-12-17    1441.662
2012-12-18    1460.691
2012-12-19    1441.022
2012-12-20    1380.220
2012-12-21    1317.366
2012-12-22    1136.842
2012-12-23    1031.437
2012-12-24     990.706
2012-12-25     899.827
2012-12-26     909.618
2012-12-27    1068.617
2012-12-28    1109.424
2012-12-29    1014.550
2012-12-30     936.786
2012-12-31     969.810
Name: Consumption, dtype: float64